In [1]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
from keras.applications.vgg16 import decode_predictions
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, BatchNormalization, Dropout, Flatten, merge, Reshape, Activation
from keras.models import Sequential, Model
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [2]:
import numpy as np
import pandas as pd
import os
import cv2


In [3]:
train=pd.read_csv('train.csv')
train.head()

,ID,Class
0,377.jpg,MIDDLE
1,17814.jpg,YOUNG
2,21283.jpg,MIDDLE
3,16496.jpg,YOUNG
4,4487.jpg,MIDDLE


In [4]:
test=pd.read_csv('test.csv')
test.head()

,ID
0,25321.jpg
1,989.jpg
2,19277.jpg
3,13093.jpg
4,5367.jpg


In [5]:
def image_read(imglist,path):
    image_array=[]
    for i in imglist:
        image=cv2.imread(path+i)
        image=cv2.resize(image,(64,64))
        image_array.append(image)
    image_array=np.array(image_array)
    return image_array

In [6]:
train_img_path="Train/"
train_imagelist=train['ID']

In [7]:
X_train=image_read(train_imagelist,train_img_path)

In [8]:
X_train.shape

(19906, 64, 64, 3)

In [9]:
test_img_path="Test/"
test_imagelist=test['ID']
X_test=image_read(test_imagelist,test_img_path)

In [10]:
X_test.shape


(6636, 64, 64, 3)

In [11]:
Y_train=train['Class']
Y_train = Y_train.map({'YOUNG': 0, 'MIDDLE': 1, 'OLD': 2})

In [12]:
from sklearn.preprocessing import LabelBinarizer 
label_binarizer = LabelBinarizer()
label_binarizer.fit(Y_train)

Y_train=label_binarizer.transform(Y_train)


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_valid, Y_train, Y_valid=train_test_split(X_train,Y_train,test_size=0.15, random_state=42)

In [14]:
X_train=X_train/255

In [15]:
X_valid=X_valid/255

In [16]:
X_test=X_test/255

In [17]:
from keras import applications

In [18]:
model = applications.VGG16(weights = "imagenet", include_top=False, input_shape = (64, 64, 3))


58892288/58889256 [==============================] - 4s 0us/step


In [19]:
for layer in model.layers:
    layer.trainable = True

In [20]:
x = model.output



x = Flatten()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.4)(x)
x = Dense(64, activation="relu")(x)
predictions = Dense(3, activation="softmax")(x)

In [21]:
model_final = Model(input = model.input, output = predictions)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [22]:
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

### loading already trained weight with same  architecture as described above

In [23]:
model_final.load_weights('weights.best.eda.hdf5')

In [24]:
model_final.compile(loss = "categorical_crossentropy",  optimizer ='adam', metrics=["accuracy"])


In [25]:
checkpointer = ModelCheckpoint(filepath='weights.best.eda.hdf5', verbose=1, save_best_only=True)


In [32]:
model_final.fit(X_train, Y_train, validation_data=(X_valid, Y_valid), epochs=2, batch_size=128, callbacks=[checkpointer], verbose=1)

Train on 16920 samples, validate on 2986 samples
Epoch 1/2
16920/16920 [==============================] - 99s 6ms/step - loss: 0.1580 - acc: 0.9450 - val_loss: 0.6155 - val_acc: 0.8034
Epoch 2/2
16920/16920 [==============================] - 99s 6ms/step - loss: 0.2058 - acc: 0.9258 - val_loss: 0.5765 - val_acc: 0.8071


In [33]:
pred=model_final.evaluate(X_train,Y_train)
print("Accuracy : " +str(pred[1]*100))
print("Total Loss  " +str(pred[0]*100))

16920/16920 [==============================] - 35s 2ms/step
Accuracy : 94.3853427896
Total Loss  14.8870170411


In [34]:
predictions = model_final.predict(X_test)
predictions = np.argmax(predictions, axis= 1)

In [35]:
classes={0: 'YOUNG', 1: 'MIDDLE', 2: 'OLD'}

In [36]:
predicted_class=[classes[x] for x in predictions]

In [37]:
predicted_class[0:3]

['MIDDLE', 'YOUNG', 'YOUNG']

In [38]:
sub = pd.DataFrame({
    "Class": predicted_class,
    "ID": test['ID']
})

In [39]:
sub.to_csv("vggpredicted_class.csv", index=False)